In [ ]:
import numpy as np
from numpy.linalg import norm
from matplotlib import pyplot as plt
from typing import Tuple, Mapping
import numba
import timeit

### Функция Бута: $f(x, y)=(x + 2y - 7)^2 + (2x + y - 5)^2$

### Глобальный минимум: $f(1, 3) = 0$

### Метод поиска: $-10 \leq x, y \leq 10$

In [ ]:
def func_but(x, y):
    return (x + 2 * y - 7) * (x + 2 * y - 7) + (2 * x + y - 5) * (2 * x + y - 5) # исходная функция 

def dx_but(x, y):
    return 10 * x + 8 * y - 34 # производная по х

def dy_but(x, y):
    return 10 * y + 8 * x - 38 # производня по у

### Реализация функции Бута для numba

In [ ]:
@numba.njit(fastmath=True)
def f(x, y):
    return (x + 2 * y - 7) * (x + 2 * y - 7) + (2 * x + y - 5) * (2 * x + y - 5) # исходная функция 

@numba.njit(fastmath=True)
def dx(x, y):
    return 10 * x + 8 * y - 34 # производная по х

@numba.njit(fastmath=True)
def dy(x, y):
    return 10 * y + 8 * x - 38 # производня по у

### Шедулер ака Pytorch StepLR

In [ ]:
class lr_scheduler():
    def __init__(self, step_size:int=5000, gamma:float=0.1, lr:float=0.1):
        '''
        step_size -- количество итераций необходимое для уменьшения скорости обучения
        gamma -- коэффициент на который домнажается скорость обучеия каждые step_size шагов
        lr -- скорость обучения
        '''
        self.iter  = 0 # переменная для подсчета шагов
        self.step_size = step_size
        self.gamma = gamma
        self.lr = lr

    def step(self):
        self.iter += 1
        if self.iter == self.step_size: # когда число итераций становится равно заданному порогу -> уменьшаем скорость обучения
            self.lr = self.lr * self.gamma
            self.iter = 0
        return self.lr

### Реализация алгоритма градиентного спуска

In [ ]:
def GD(f:Mapping, dx:Mapping, dy:Mapping, x0:np.ndarray, y0:np.ndarray, lr_start:float=0.001, iter=100, thr:np.ndarray=None):
    '''
        f -- исходная функция
        dx -- производня по х от исходной функции
        dy -- производня по у от исходной функции
        x0 -- стартовая точка по координате х
        y0 -- стартовая точка по координате у
        lr_start -- изначальное значение скорости обучения
        iter -- число итераций выполнения алгоритма
        thr -- точность вычислений 
    '''
    x_old = x0.copy()
    y_old = y0.copy()
    
    scheduler = lr_scheduler(lr=lr_start)
    for i in range(iter):
        if np.linalg.norm(dx(x_old, y_old)) > thr[0] and np.linalg.norm(dy(x_old, y_old)) > thr[0]:
            gradX = dx(x_old, y_old)
            gradY = dy(x_old, y_old)
            x_old = x_old - scheduler.lr * gradX
            y_old = y_old - scheduler.lr * gradY
            scheduler.step()
    plt.show()
    return x_old, y_old

### Реализация алгоритма градиентного спуска с использованием Numba

In [ ]:
@numba.njit(fastmath=True)
def GD_numba(x0:np.ndarray, y0:np.ndarray, lr_start:float=0.001, iter:np.ndarray=100, thr:np.ndarray=None, step_size:int=5000, gamma:float=0.1):
    '''
        x0 -- стартовая точка по координате х
        y0 -- стартовая точка по координате у
        lr_start -- изначальное значение скорости обучения
        iter -- число итераций выполнения алгоритма
        thr -- точность вычислений
        step_size -- колличество шагов необходимоее для уменьшения скорости обучения
        gamma -- коэффициент уменьшения скорости обучения
    '''
    x_old = x0.copy()
    y_old = y0.copy()
    t = thr.copy()
    iteration = 0
    while iter[0] > 0:
        a = np.array([dx(x_old[0], y_old[0])])
        if norm(np.array([dx(x_old[0], y_old[0])])) > thr[0] and norm(np.array([dy(x_old[0], y_old[0])])) > thr[0]:
            gradX = dx(x_old, y_old)
            gradY = dy(x_old, y_old)
            x_old = x_old - lr_start * gradX
            y_old = y_old - lr_start * gradY
            iteration += 1
            if iter == step_size:
                iteration = 0
                lr_start = lr_start * gamma
        iter -= 1
    return x_old, y_old

### Результаты и время работы обысного алгоритма

In [ ]:
x, y = GD(f=func_but, dx=dx_but, dy=dy_but, x0=np.array([-1]), y0=np.array([-1]), iter=10000, thr=np.array([1e-10]))
print(f"Начальная точка: [{-1}], [{-1}]")
print(f'Точка найденного минимума: {x}, {y}')
print(f'Значение функции в найденной точке: {func_but(x, y)}')
%timeit -n100 GD(f=func_but, dx=dx_but, dy=dy_but, x0=np.array([-1]), y0=np.array([-1]), iter=100, thr=np.array([10]))

Начальная точка: [-1], [-1]
Точка найденного минимума: [1.00001653], [2.99998347]
Значение функции в найденной точке: [5.4672261e-10]
3.45 ms ± 475 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Результаты и время работы алгоритма ускоренного Numba

##### (отвык я от типизации, было очень трудно втянуться и зафиксить все ворнинги и ошибки)

In [ ]:
x, y = GD_numba(x0=np.array([-1.0]), y0=np.array([-1.0]), iter=np.array([10000]), thr=np.array([1e-10]))
print(f"Начальная точка: [{-1}], [{-1}]")
print(f'Точка найденного минимума: {x}, {y}')
print(f'Значение функции в найденной точке: {func_but(x, y)}')
%timeit -n100 GD_numba(x0=np.array([-1.0]), y0=np.array([-1.0]), iter=np.array([300]), thr=np.array([1e-10]))

Начальная точка: [-1], [-1]
Точка найденного минимума: [1.0000165], [2.9999835]
Значение функции в найденной точке: [5.44755788e-10]
734 µs ± 51.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
